# Bigram

In [1]:
# Download input data
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-03-17 10:36:21--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  5.62MB/s    in 0.2s    

2023-03-17 10:36:22 (5.62 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [113]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import IPython
from tqdm import tqdm

In [5]:
# Read the input file
with open('./input.txt', 'r', encoding='utf-8') as f:
    text = f.read() # should be simple plain text file
print('corpus length:', len(text))
print(text[:100])

corpus length: 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [8]:
# Create character-level vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print('total chars:', vocab_size)
print(''.join(chars))

total chars: 65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [10]:
# Tokenize the text
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[ch] for ch in x])

print('encoded:', encode("hey there!"))
print('decoded:', decode(encode("hey there!")))

encoded: [46, 43, 63, 1, 58, 46, 43, 56, 43, 2]
decoded: hey there!


In [16]:
# Encode the entire dataset
data = torch.tensor(encode(text))
print(data.shape, data.dtype)
print(decode(data[:100].tolist()))

torch.Size([1115394]) torch.int64
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [23]:
# Split dataset into train and validation dataset
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]
len(train_data), len(val_data)

(1003854, 111540)

In [43]:
# When we train a transformer we train with randomly sampled chunks of the text data (block_size)
torch.manual_seed(1337)
block_size = 8 # We still train with all smaller context sizes but this is the maximum. 0 --> 1, 0,1 --> 2, 0,1,2 -->3 ... 0...n-1 --> n 
batch_size = 4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([train_data[i:i+block_size] for i in ix])
    y = torch.stack([train_data[i+1:i+block_size+1] for i in ix])
    return x, y
xb, yb = get_batch('train')
print('xb', xb)
print('yb', yb)

# Illustrate the training set
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        # print(f"when input is {context.tolist()} the target: {target}")

xb tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
yb tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [85]:
class BigramLanguageModel(nn.Module):
    """Bigram LM because we only look at one character of context to make predictions for the next char."""
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (Batch,Time,Channel), acting as a lookup table for the probabilities of the next word
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1) # final dimension as that's the dimension with the activation values.
            idx_next = torch.multinomial(probs, num_samples=1, replacement=False)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size)

# Sample input
input = torch.zeros((1,1), dtype=torch.long)
out = decode(m.generate(input, max_new_tokens=100)[0].tolist()) # [0] is needed as we have a Batch dimension
print(out)

tensor([[0]])

RVYZAEV.juadQMh:ntSrFZXjAgQVJGRYHHjSfQP&.sZAOg:Kty
CT.srvL$ RYbjAqb?hB,dUCoQny
Ovtuj,QV-eOgY,ugOuVZA


In [90]:
# Optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [105]:
batch_size = 32
for steps in tqdm(range(10000)):
    # get batch
    xb, yb = get_batch('train')

    # forward pass
    logits, loss = m(xb, yb)

    # backward pass
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


100%|██████████| 10000/10000 [00:06<00:00, 1617.57it/s]

2.440438985824585


In [107]:
# Sample input
input = torch.zeros((1,1), dtype=torch.long)
out = decode(m.generate(input, max_new_tokens=500)[0].tolist()) # [0] is needed as we have a Batch dimension
print(out)


S:
EROLOUSTI nouppery ANCI rass,
Weatheddeerean henowens, my ndeb kersade
The us d ouleasthantous warer is wio
CENorshasod,
LO at l hafr d O ou sd s QUKI the ve sy INICORINRBurcofthecot inalare s ss ngaimpe windyothamet hakil in? whe tourpr sth yonofand h yonghy, t loutou fous's asllllaicorpey;
CLouriour; lid n, onon, outift He til a as wnan
CENGAn toucy o o s ake, g gt I ared Austhannghe t
I whe g Nors, wothearetiolerere ady, t-rd;

Sargar t, deswile Is
Whisid onou hy Y:
e, a ppr t:
TENCHIAGemy


# Exploring self attention

In [114]:
B, T, C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [116]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # outputs, (T, C)
        xbow[b,t] = torch.mean(xprev, dim=0) # mean over C values, reducing the dimension of T.

In [125]:
# Version 2: Modifying the tril
wei = torch.tril(torch.ones(T, T))
wei /= torch.sum(wei, dim=1, keepdim=True)
xbow2 = wei @ x
# print(a)
# print(b)
# print(c)
torch.allclose(xbow, xbow2)

True

In [133]:
# Version 3: with softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill_(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
tril, wei, torch.allclose(xbow, xbow3)

(tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1.]]),
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
         [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
         [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]]),
 True)

In [144]:
# Version 4: self attention
"""Gather information in the past in a data dependent one

Every token in each position will emit a query and key vector.
Query = what am i looking for
Key = what do i contain?
Value = what information from the current token should i use?

wei is the dot product of the query vector and all the other key vectors
value filters out the current token

Theory:
- Attention is a communication mechanism between nodes of a graph
- Elements across the batch dimension are independent and do not talk to one another.
"""
# Init
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# Single head of self attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)

wei = q @ k.transpose(-2, -1)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill_(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei @ v
# out = wei @ x

In [143]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.9186, 0.0814, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5819, 0.2927, 0.1253, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.9006, 0.0604, 0.0216, 0.0174, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0096, 0.0704, 0.2335, 0.2736, 0.4129, 0.0000, 0.0000, 0.0000],
         [0.5311, 0.2699, 0.0670, 0.0159, 0.0467, 0.0694, 0.0000, 0.0000],
         [0.0662, 0.4095, 0.2338, 0.0319, 0.0795, 0.0756, 0.1034, 0.0000],
         [0.5785, 0.0555, 0.0236, 0.1225, 0.1329, 0.0643, 0.0128, 0.0099]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3950, 0.6050, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4125, 0.1096, 0.4779, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0043, 0.3167, 0.1938, 0.4852, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0149, 0.0236, 0.3224, 0.5535, 0.0856, 0.0000, 0.0000, 0.0000],
         [0.0541, 0.152

# LayerNorm

In [147]:
class LayerNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]
  
x = torch.randn(32, 100)
module = LayerNorm1d(100)
out = module(x)